In [1]:
import numpy as np
from keras.models import load_model
import cv2
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from time import sleep

Using TensorFlow backend.


In [2]:
#Loading data and clasifier model

face_classifier = cv2.CascadeClassifier('D:\\Deep learning dataset\\haarcascade_frontalface_default.xml')
classifier = load_model('Facial_Expression_Recognition.hdf5')

#Labels for prediction
class_labels = ["Angry", "Happy", "Neutral", "Sad", "Surprise"]

#video streaming
cap = cv2.VideoCapture(0)

while True:
    #Readig the video into frame
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray,scaleFactor=  1.3, minNeighbors = 5,flags=cv2.CASCADE_SCALE_IMAGE)
       
    for (x,y,w,h) in faces:
        #Drawing Rectangle on face
        cv2.rectangle(gray, (x,y),(x+w, y+h),(255,0,0),2)
        
        #Extracting ROI of the face from the grayscale image
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48))
    
        if np.sum(roi_gray)!= 0: #If there is atleast one face in frame
            roi = roi_gray.astype('float')/255
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis =0)        
        
            #Predicting the faceial expression
            pred = classifier.predict(roi)[0]
            label = class_labels[pred.argmax()]
            #Getting position of label
            label_position = (x,y)
            
            #putting text on face rectangle drawn above
            cv2.putText(gray, label, label_position,cv2.FONT_HERSHEY_SIMPLEX, 2,(0,255,255), 3)
    
        else:
        
            cv2.putText(gray, "no face", (20,60),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,0), 3)
        
        cv2.imshow("Emotion", gray) 
   
           
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()